In [72]:
from tqdm.notebook import tqdm

import pandas as pd 
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

In [73]:
from sklearn.model_selection import train_test_split

from gensim.models import Word2Vec
from gensim.models.phrases import Phrases, Phraser

pd.set_option('display.max_colwidth', None)
from sklearn.metrics import f1_score,make_scorer

In [74]:
import multiprocessing
from time import time 
import pickle

In [75]:
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.neural_network import MLPClassifier
import catboost as cb

Загрузка датафрейма

In [76]:
df=pd.read_csv(r'D:\Python_Projects\hakaton\dataset\emotional_dataset_full.csv')
df.dropna(inplace=True)
df.head()

,Unnamed: 0,text,target,prepared_text,prepared_text_lemm,prepared_text_lemm_nostop
0,0,"Да, все-таки он немного похож на него. Но мой мальчик все равно лучше:D",1,да все таки он немного похож на него но мой мальчик все равно лучше :D,да все таки он немного похожий на него но мой мальчик все равный хороший :d,таки немного похожий мальчик равный хороший :d
1,1,RT @KatiaCheh: Ну ты идиотка) я испугалась за тебя!!!,1,ну ты идиотка я испугалась за тебя ),ну ты идиотка я испугаться за тебя ),идиотка испугаться )
2,2,"RT @digger2912: ""Кто то в углу сидит и погибает от голода, а мы ещё 2 порции взяли, хотя уже и так жрать не хотим"" :DD http://t.co/GqG6iuE2…",1,кто то в углу сидит и погибает от голода а мы ещё порции взяли хотя уже и так жрать не хотим :D,кто то в угол сидеть и погибать от голод а мы ещё порция взять хотя уже и так жрать не хотеть :d,угол сидеть погибать голод ещё порция взять хотя жрать хотеть :d
3,3,"@irina_dyshkant Вот что значит страшилка :D\nНо блин,посмотрев все части,у тебя создастся ощущение,что авторы курили что-то :D",1,вот что значит страшилка но блин посмотрев все части у тебя создастся ощущение что авторы курили что то :D :D,вот что значить страшилка но блин посмотреть весь часть у тебя создаться ощущение что автор курить что то :d :d,значить страшилка блин посмотреть весь часть создаться ощущение автор курить :d :d
4,4,ну любишь или нет? — Я не знаю кто ты бля:D http://t.co/brf9eNg1U6,1,ну любишь или нет я не знаю кто ты бля :D,ну любить или нет я не знать кто ты бля :d,любить знать бля :d


### Создание Биграмм.

Phrases используется для автоматического распознавания распространенных фраз

Phraser же сократить потребление времени Phrases

In [77]:
sent = [row.split() for row in df['prepared_text_lemm_nostop']]
phrases = Phrases(sent, min_count=30, progress_per=10000)

" sent = [row.split() for row in df['prepared_text_lemm_nostop']]\nphrases = Phrases(sent, min_count=30, progress_per=10000) "

In [78]:
bigram = Phraser(phrases)
sentences = bigram[sent]

' bigram = Phraser(phrases)\nsentences = bigram[sent] '

### Построение embedding-ов с помощью World2Vec.

1. Настройка параметров World2Vec
2. Построение словаря

In [79]:
cores = multiprocessing.cpu_count() #количество ядер CPU
print(cores)

' cores = multiprocessing.cpu_count() #количество ядер CPU\nprint(cores) '

Инициализация модели World2Vec:
+ min_count - игнорирование слов с частотой меньше заданной
+ window - максимальное расстояние между текущим и предсказанным словом
+ vector_size - размер признаков
+ sample - порог для уменьшение выборки часто встречающихся слов
+ alpha - скорость обучения
+ min_alpha - скорость до которой будет постепенно снижаться скорость обучения
+ negative - выборка для заглушения указанного количества шумных слов
+ workers - количество потоков для работы

In [80]:
w2v_model = Word2Vec(min_count=20,
                     window=2,
                     vector_size=300,
                     sample=6e-5, 
                     alpha=0.03, 
                     min_alpha=0.0007, 
                     negative=20,
                     workers=cores-1)


' w2v_model = Word2Vec(min_count=20,\n                     window=2,\n                     vector_size=300,\n                     sample=6e-5, \n                     alpha=0.03, \n                     min_alpha=0.0007, \n                     negative=20,\n                     workers=cores-1) '

Построение словаря

In [81]:
t = time()
w2v_model.build_vocab(sentences, progress_per=10000)
print('Time to build vocab: {} mins'.format(round((time() - t) / 60, 2)))

" t = time()\nw2v_model.build_vocab(sentences, progress_per=10000)\nprint('Time to build vocab: {} mins'.format(round((time() - t) / 60, 2))) "

Тренировка модели World2Vec

In [82]:
t = time()
w2v_model.train(sentences, 
                total_examples=w2v_model.corpus_count, 
                epochs=30, 
                report_delay=1)
print('Time to train the model: {} mins'.format(round((time() - t) / 60, 2)))

" t = time()\nw2v_model.train(sentences, \n                total_examples=w2v_model.corpus_count, \n                epochs=30, \n                report_delay=1)\nprint('Time to train the model: {} mins'.format(round((time() - t) / 60, 2))) "

In [83]:
w2v_model.init_sims(replace=True) #замораживаем сеть для повышение эффективности памяти

' w2v_model.init_sims(replace=True) #замораживаем сеть для повышение эффективности памяти '

In [84]:
import pickle 
pkl_filename = "emotion_word2vec.pkl" 
with open(pkl_filename, 'wb') as file: 
    pickle.dump(w2v_model, file)

' import pickle \npkl_filename = "emotion_word2vec.pkl" \nwith open(pkl_filename, \'wb\') as file: \n    pickle.dump(w2v_model, file) '

Тест обученной модели

покажем самые близкие по значению к заданному слову

In [85]:
w2v_model.wv.most_similar(positive=["дом"])

' w2v_model.wv.most_similar(positive=["дом"]) '

### Подготовка к обучению
Загрузим обученную модель

In [86]:
with open(r'D:\Python_Projects\hakaton\emotion_word2vec.pkl', 'rb') as file:
    w2v_model = pickle.load(file)

функция для векторизации с помощью word2vec

In [87]:
def text_to_vector(text):
    words = text.split()
    vectors = [w2v_model.wv[word] if word in w2v_model.wv else np.zeros(w2v_model.vector_size) for word in words]
    return np.mean(vectors, axis=0)

df['vectorized_text'] = df['prepared_text_lemm_nostop'].apply(text_to_vector)

In [88]:
df['vectorized_text'] = df['vectorized_text'].apply(lambda x: np.array(x))

Создание тренировочной и тестовой выборок

In [89]:
RANDOM_STATE=42

In [90]:
X = df['vectorized_text']
y = df['target']

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, stratify=y, random_state=RANDOM_STATE)

### Обучение моделей
**Логистическая Регрессия**

In [20]:
logreg = LogisticRegression(solver='liblinear', max_iter=500,penalty='l1',
                              random_state=RANDOM_STATE,C=1)
logreg.fit(list(X_train), y_train)

LogisticRegression(C=1, max_iter=500, penalty='l1', random_state=42,
                   solver='liblinear')

In [22]:
y_pred=logreg.predict(list(X_test))
print('Значение метрики для Логистической Регрессии:', f1_score(y_pred,y_test, average='macro'))

Значение метрики для Логистической Регрессии: 0.9137872241159792


In [24]:
import pickle 
pkl_filename = "logreg_emotion_word2vec.pkl" 
with open(pkl_filename, 'wb') as file: 
    pickle.dump(logreg, file) 

**KNN**

In [25]:
knn = KNeighborsClassifier(algorithm='auto',metric='cosine',weights='uniform',n_neighbors=5,n_jobs=-1)
knn.fit(list(X_train), y_train)

KNeighborsClassifier(metric='cosine', n_jobs=-1)

In [26]:
y_pred=knn.predict(list(X_test))
print('Значение метрики для Метода ближайших соседей:', f1_score(y_pred,y_test, average='macro'))

Значение метрики для Метода ближайших соседей: 0.9258015300887177


In [27]:
pkl_filename = "knn_emotion_word2vec.pkl" 
with open(pkl_filename, 'wb') as file: 
    pickle.dump(knn, file)

**Наивный Байес**

In [28]:
gauss = GaussianNB()
gauss.fit(list(X_train), y_train)

GaussianNB()

In [29]:
y_pred=gauss.predict(list(X_test))
print('Значение метрики для Наивного Байеса:', f1_score(y_pred,y_test, average='macro')) 

Значение метрики для Наивного Байеса: 0.8916678257819605


In [30]:
pkl_filename = "gauss_emotion_word2vec.pkl" 
with open(pkl_filename, 'wb') as file: 
    pickle.dump(gauss, file)

**LDA**

In [31]:
lda = LinearDiscriminantAnalysis()
lda.fit(list(X_train), y_train)

LinearDiscriminantAnalysis()

In [32]:
y_pred=lda.predict(list(X_test))
print('Значение метрики для LDA:', f1_score(y_pred,y_test, average='macro'))  

Значение метрики для LDA: 0.8804279900672412


In [33]:
pkl_filename = "lda_emotion_word2vec.pkl" 
with open(pkl_filename, 'wb') as file: 
    pickle.dump(lda, file)

**CatBoost**

In [36]:
catb = cb.CatBoostClassifier(iterations=100, learning_rate=0.1, depth=10)
catb.fit(list(X_train), y_train)

0:	learn: 0.9738152	total: 10.3s	remaining: 16m 58s
1:	learn: 0.8776496	total: 20.1s	remaining: 16m 25s
2:	learn: 0.8000808	total: 29.9s	remaining: 16m 7s
3:	learn: 0.7352268	total: 39.6s	remaining: 15m 49s
4:	learn: 0.6822813	total: 48.7s	remaining: 15m 24s
5:	learn: 0.6370423	total: 58.4s	remaining: 15m 14s
6:	learn: 0.5972868	total: 1m 8s	remaining: 15m 6s
7:	learn: 0.5631092	total: 1m 17s	remaining: 14m 50s
8:	learn: 0.5324838	total: 1m 26s	remaining: 14m 34s
9:	learn: 0.5053022	total: 1m 35s	remaining: 14m 19s
10:	learn: 0.4818284	total: 1m 45s	remaining: 14m 13s
11:	learn: 0.4606576	total: 1m 56s	remaining: 14m 11s
12:	learn: 0.4415384	total: 2m 7s	remaining: 14m 15s
13:	learn: 0.4250253	total: 2m 18s	remaining: 14m 9s
14:	learn: 0.4096549	total: 2m 28s	remaining: 14m 1s
15:	learn: 0.3959254	total: 2m 39s	remaining: 13m 57s
16:	learn: 0.3832261	total: 2m 50s	remaining: 13m 52s
17:	learn: 0.3717626	total: 2m 59s	remaining: 13m 37s
18:	learn: 0.3613672	total: 3m 9s	remaining: 13m 2

In [37]:
y_pred=catb.predict(list(X_test))
print('Значение метрики для CatBoost:', f1_score(y_pred,y_test, average='macro'))

Значение метрики для CatBoost: 0.9349563900219494


In [38]:
pkl_filename = "catb_emotion_word2vec.pkl" 
with open(pkl_filename, 'wb') as file: 
    pickle.dump(catb, file)

**MLP**

In [91]:
mlp = MLPClassifier(hidden_layer_sizes=(5, 5, 5,5,5), max_iter=100, solver='sgd', activation='relu')
mlp.fit(list(X_train), y_train)

c:\Users\Dasha\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (100) reached and the optimization hasn't converged yet.
  warnings.warn(


MLPClassifier(hidden_layer_sizes=(5, 5, 5, 5, 5), max_iter=100, solver='sgd')

In [93]:
y_pred=mlp.predict(list(X_test))
print('Значение метрики для MLP:', f1_score(y_pred,y_test, average='macro'))

Значение метрики для MLP: 0.942869693174034


In [94]:
pkl_filename = "mlp_emotion_word2vec.pkl" 
with open(pkl_filename, 'wb') as file: 
    pickle.dump(mlp, file)

### Создание тестового csv с ответами модели

загрузка датафрейма

In [95]:
df_test=pd.read_csv(r'D:\Python_Projects\hakaton\dataset\emotional_dataset_full.csv')
df_test.dropna(inplace=True)
df_test.head()

,Unnamed: 0,text,target,prepared_text,prepared_text_lemm,prepared_text_lemm_nostop
0,0,"Да, все-таки он немного похож на него. Но мой мальчик все равно лучше:D",1,да все таки он немного похож на него но мой мальчик все равно лучше :D,да все таки он немного похожий на него но мой мальчик все равный хороший :d,таки немного похожий мальчик равный хороший :d
1,1,RT @KatiaCheh: Ну ты идиотка) я испугалась за тебя!!!,1,ну ты идиотка я испугалась за тебя ),ну ты идиотка я испугаться за тебя ),идиотка испугаться )
2,2,"RT @digger2912: ""Кто то в углу сидит и погибает от голода, а мы ещё 2 порции взяли, хотя уже и так жрать не хотим"" :DD http://t.co/GqG6iuE2…",1,кто то в углу сидит и погибает от голода а мы ещё порции взяли хотя уже и так жрать не хотим :D,кто то в угол сидеть и погибать от голод а мы ещё порция взять хотя уже и так жрать не хотеть :d,угол сидеть погибать голод ещё порция взять хотя жрать хотеть :d
3,3,"@irina_dyshkant Вот что значит страшилка :D\nНо блин,посмотрев все части,у тебя создастся ощущение,что авторы курили что-то :D",1,вот что значит страшилка но блин посмотрев все части у тебя создастся ощущение что авторы курили что то :D :D,вот что значить страшилка но блин посмотреть весь часть у тебя создаться ощущение что автор курить что то :d :d,значить страшилка блин посмотреть весь часть создаться ощущение автор курить :d :d
4,4,ну любишь или нет? — Я не знаю кто ты бля:D http://t.co/brf9eNg1U6,1,ну любишь или нет я не знаю кто ты бля :D,ну любить или нет я не знать кто ты бля :d,любить знать бля :d


In [96]:
df_test = df_test.sample(10000)

In [97]:
df_test['target'].value_counts()

target
 0    4854
 1    2635
-1    2511
Name: count, dtype: int64

Подготовка к предсказанию

In [98]:
with open(r'D:\Python_Projects\hakaton\emotion_word2vec.pkl', 'rb') as file:
    w2v_model = pickle.load(file)

def text_to_vector(text):
    words = text.split()
    vectors = [w2v_model.wv[word] if word in w2v_model.wv else np.zeros(w2v_model.vector_size) for word in words]
    return np.mean(vectors, axis=0)

df_test['vectorized_text'] = df_test['prepared_text_lemm_nostop'].apply(text_to_vector)
df_test['vectorized_text'] = df_test['vectorized_text'].apply(lambda x: np.array(x))

Предсказание

In [99]:
with open(r'D:\Python_Projects\hakaton\mlp_emotion_word2vec.pkl', 'rb') as file:
    mlp = pickle.load(file)

In [100]:
pred = mlp.predict(list(df_test['vectorized_text']))

In [101]:
df_test['predict'] = pred

Просмотр результата

In [103]:
df_test[['target','predict','text']].sample(10)

,target,predict,text
258319,0,0,молодец супер! 👏👏👏! весь в деда!! пригласите его на праздник в шереметьево! пусть порадует своими песнями! 😄👍
245231,0,0,"с комментариями со всеми согласна, но мне кажется хозяйка в висячем положении (прилеплена)"
67288,1,1,"""Хватит плакать, тряпка!""\n © Вилка:D"
87852,1,1,"Небольшой плакатик в школу;) / С наступающим, ребятки! http://t.co/FR6YUf7xwh"
93648,1,1,"@tonn_pavloff @komzov много мест России перечислено :)так где были то? )вы же не в вакууме живете, а в городе, чиновники таже часть России"
94648,1,1,"@Apelsinka___ ну да, этим ребятам я не завидую. Раньше, пока мы сети не объединили, они по нескольку раз в день с документами ходили :)"
25660,1,1,"RT @rokihobymaxy: все, теперь я могу смело сказать: ""Я свободен!""... или спеть :)"
89182,1,1,Эти медведи не стоят 12 крышак!инфа спешел фо Порягина;)
114312,-1,-1,RT @Kimelfeeeld: сесть на диету - да!\nпозавтракать овсянкой и яблоком - да!\nне есть ничего лишнего - да! \nзапечь картошку с салом - да!(
18040,1,1,"Подарочный день) у кого то палочки а у меня доска, которую давно хотела, спасибо кот http://t.co/vkcmpAWQfZ"


In [105]:
print('Исходный размер датафрейма:', df_test.shape[0])
print('Правильные ответы:', df_test[df_test['predict']==df_test['target']].shape[0])

Исходный размер датафрейма: 10000
Правильные ответы: 9461
